# Scheduling

El <b>scheduler</b> conocido en espaniol como el <b>planificador</b> es el que se encarga de darle la ilucion al programador de que todos los programas se estan ejecutando al mismo tiempo. La realidad es muy diferente, los programas se ejecutan un <b>slice</b> de tiempo cada uno. Unos con mas priodidad que otros.

Asi que es el <b>scheduler</b> el que decide cual es el siguiente programa a ejecutar y cual sera el slice de tiempo que le asignara.

<hr>

## Metricas

### Turnaround Time

\begin{equation} 
\large T_{turnaround} = T_{completion} - T_{arrival}
\end{equation}

<b>Observacion</b>: Esta es una <b>metrica</b> de <b>performance</b>.

Donde:
- <b>Time completion</b>: Es el tiempo que se tarda en completar la tarea.
- <b>Time arrival</b>: Es el tiempo que tarda en arrivar la tarea.

### Response Time

El tiempo que que tarda una tarea desde que llego hasta que es corrida por el scheduler por primera ves.

\begin{equation} 
\large T_{response} = T_{firstrun} - T_{arrival}
\end{equation}

Donde:
- <b>Time firstrun</b>: Es el tiempo que en ser corrida por primera ves por el scheduler.
- <b>Time arrival</b>: Es el tiempo que tarda en arrivar la tarea.

<hr>

## Politicas 

## First In, First Out (FIFO)

Esta es la politica mas simple que existe, es simplemente que el scheduler procese los programas por orden de llegada y uno a la ves. Osea que hasta que no termine por completo de ejecutar uno, no siga con el siguiente.

Veamos un <b>ejemplo</b>:

Tenemos tres programas el A, el B y el C. Llega primero el A, luego el B, y por ultimo llega el C (pero aproximadamente al mismo tiempo => time arrival es 0 en los 3). Entonces como se ejecutan por orden de llegada pasa lo siguiente:

<img src="Imagenes/scheduler_fifo.png">

El A tarda 10 segundos y el B tarda lo que tardo el A mas lo que tardo el B y por ultimo el C tarda lo que tardo el A mas lo que tardo el B mas lo que tardo el C. Entonces el A tardo 10 segundos, el B 20 segundos y el C tardo 30 segundos. Por lo tanto el <b>turnaround time</b> promedio es (10 + 20 + 30) / 3 = 60 / 3 = 20 segundos. 

En este caso todos los programas tardaron lo mismo en ejecurse, el tema es cuando el que llega primero es el que mas tarda en ejecutarse, ahi surge un problema y es que el <b>turnaround time</b> promedio aumenta mucho porque los siguientes programas tuvieron que esperar que el primero que es el mas tarda, termine. Este es el efecto <b>Convoy</b>.

<img src="Imagenes/sheduler_convoy_effect.png">

Tiempo promedio: (100 + 110 + 120) / 3 = 110

## Shortest Job First (SJF)

Esta politica se tiene la mejora de que ejecuta los programas en orden de lo que tardan, de menor a mayor. Esto evita el efecto Convoy.

<img src="Imagenes/scheduler_shortest_job_first.png">

<b>turnaround time</b>: (10 + 20 + 120) / 3 = 50

Ahora, acerquemosnos un poc mas a la realidad, todos los programas no tienen porque llegar al mismo tiempo. Supongamos que primero llega el A en t = 0, y luego llega el B y el C en t = 10. Pasaria lo siguiente.

<img src="Imagenes/scheduler_shortest_job_first_2.png">

Por que el B y el C tarden menos, como el A llego primero lo comenzo a ejecutar y segun esta politica no puede ejecutar un context switch por lo tanto, el <b>time turnaround</b> es:

\begin{equation} 
\large T_{turnaround} = \frac{(100 - 0) + (110 - 10) + (120 - 10)}{3} = 103.3
\end{equation}

Es bastante, asi que vamos a tener que seguir implentando mejoras.

## Shortest Time-To-Completion First (STCF)

Esta politica habilita el uso del <b>context switch</b>. Sigamos con el ejemplo anterior para verlo.

<img src="Imagenes/scheduler_shortest_job_first_3.png">

Las tareas llegan en el mismo orden y tiempo que el ejemplo anterior. Entonces comienza a ejecutar la tarea A, luego llegan la B y la C. Como esta politica tiene la habilidad de hacer context switch, y le interesa dar al usuario un mejor <b>response time</b> va a correr la tarea B y luego la C que tienen menor <b>time to completion</b> es decir son tareas mas cortas. Veamos como mejora el <b>time turnaround</b>:

\begin{equation} 
\large T_{turnaround} = \frac{(120 - 0) + (20 - 10) + (30 - 10)}{3} = 50
\end{equation}

## Round Robin (RR)

Esta politica introduce una significativa mejora en el <b>response time</b>. Ya que no espera a ejecutar una tarea por completo para pasar a la siguiente, sino que ejecuta un cachito de tiempo cada tarea, esto hace que el usuario valla teniendo feedback. A este cachito de tiempo se le llama <b>time slice</b>, tambien conocido como <b>scheduling quantum</b>. Por esta razon es que a Round Robin a veces se le llama <b>Time Slicing</b>

<img src="Imagenes/scheduler_round_robin.png">

Veamos con un <b>ejemplo</b> como mejora el <b>response time</b>. Supongamos que tenemos 3 tareas, A, B, C, todas llegan al mismo tiempo, cada una tarda en completarse 5 segundos (ver figura de arriba). Si usamos la politica SJF el <b>response time</b> es [(0 - 0) + (5 - 0) + (10 - 0)] / 3 = 5, ahora bien si usamos la politica <b>RR</b> el <b>response time</b> es (0 + 1 + 2) / 3 = 1.

Si bien el <b>response time</b> siempre mejora, el <b>turnaround time</b> en muchos casos seguramente aumente, osea empeore debido que los <b>context switch</b> tienen un costo temporal.

Asi que no se deben hacer demasiado pequenios los <b>time slice</b>. 

Para generalizar un poco la cuestion hasta aca, tenemos dos tipos de politicas las que mejoran el <b>time turnaround</b> que son <b>SJF</b> y <b>STCF</b> y las que mejoran el <b>time response</b> que es <b>RR</b>.

## Incorporating I / O

Ahora vamos a poner a poner las cosas de forma todabia mas realista, vamos a tener en cuenta que los programas tienen INPUT y OUTPUT. Esto cambia un poco las cosas por el lado de que cuando tienen que escribir o leer de algun periferico demoran mas tiempo de que si operan simplemente con RAM o los registros de procesador. Esto hace que por ejemplo si un programa tiene que leer de disco, el CPU se quede esperando hasta que el programa termine de leer. Y por lo tanto el CPU se quede sin hacer nada ese tiempo. Veamos esto con el siguiente esquema:

<img src="Imagenes/scheduler_poor_use_of_resources.png">

Asi que la solucion a esto es que el procesador en el tiempo que esta esperando algo de I/O. No se quede esperando sino que siga ejecutando otro tarea, es decir, aprobechando el tiempo al maximo, esto se ve en el siguiente esquema.

<img src="Imagenes/scheduler_overlap.png">

## The Multi-Level Feedback Queue (MLFQ)

Este algoritmo o politica de Scheduling fue descrito por <b>Corbato</b> en 1962 y fue ganador del gran premio <b>Turing Award</b>. Este tiene la meta de optimizar el <b>turnaround time</b> como lo hace SJF y SCTF, pero teniendo en encuenta la realidad, y es que no se conoce que van a hacer los procesos, como para determinar cual es el mas corto. Y tambien tienel meta de optimizar el <b>response time</b> como lo hace RR, para que los usuarios puedan interctuar con la pc de forma placentera.

La gran pregunta es entonces: <b>De que manera hacer schedule sin el perfecto conocimiento de los procesos?</b>

<b>MLFQ</b> es un sistema que aprende del pasado para predecir el futuro.